# 1. Stats

In [ ]:
%run ./scripts/stats.py -c ../ancora-3.0.1es

# 2. Etiquetado de Secuencias: Parte 2

Usaremos los scripts train.py y eval.py del práctico 1:

## 2.1. Baseline Malo: Todo es Sustantivo Singular

Primero "entrenamos" este baseline (no hace nada en realidad):

In [ ]:
%run ./scripts/train.py -c ../ancora-3.0.1es -m badbase -o badbase

Luego evaluamos. Usamos la opción '-m' para imprimir la matrz de confusión:

In [ ]:
%run ./scripts/eval.py -c ../ancora-3.0.1es -i badbase -m

La matriz de confusión muestra en la fila X columna Y la proporción de veces que el tagger eligió el tag Y pero el tag verdadero es X. Sólo se imprime la matriz para los 10 tags más frecuentes.

## 2.2. Baseline (Menos Malo): Etiqueta Más Frecuente

**Primero resolver ejercicio 2 del práctico!**

In [ ]:
%run ./scripts/train.py -c ../ancora-3.0.1es -m base -o base

In [ ]:
%run ./scripts/eval.py -c ../ancora-3.0.1es -i base -m

In [ ]:
%run ./scripts/eval.py -c ../ancora-3.0.1es -i base --heatmap

**Se puede ver que uno de los principales errores del tagger es confundir nombre propio (np00000) con nombre común (nc0s000), con un 2.05% del error total.**

In [ ]:
# Chequeado hasta aca - 17/12

## 2.3. Análisis de Errores (No estoy seguro si esto va aca o en la cuarta parte)

El script de evaluación eval.py calcula variables útiles para hacer análisis de errores:

- **error_count:** matriz de confusión completa.
- **error_sents:** matriz de índices de las oraciones en las que ocurre cada error.

Por ejemplo, podemos ver algunas oraciones en las que el tagger confunde nombre propio con nombre común:

In [ ]:
error_count['np00000']['nc0s000']  # / total da 2.05%

In [ ]:
sorted(error_sents['np00000']['nc0s000'])[:10]

Veamos la primer oración:

In [ ]:
sent = sents[0]
words, tags = zip(*sent)  # separar las palabras de los tags
sent[:10]

In [ ]:
model.tag(words[:10])

In [ ]:
model.unknown('Telecomunicaciones')

Es claro que el tagger se puede mejorar para palabras desconocidas mirando características como el uso de mayúsculas y también de caracteres no estándar del inglés (e.g. letras acentuadas).

___
## 3. Clasificación

#### Armando el train

In [1]:
from tagging.ancora import SimpleAncoraCorpusReader
from tagging.baseline import BaselineTagger, BadBaselineTagger
from classifier import feature_dict
%reload_ext classifier

models = {
    'badbase': BadBaselineTagger,
    'base': BaselineTagger,
}


# load the data
files = 'CESS-CAST-(A|AA|P)/.*\.tbf\.xml'
corpus = SimpleAncoraCorpusReader('../ancora-3.0.1es', files)
sents = corpus.tagged_sents()

In [2]:
##########################

In [3]:
from classifier import ClassifierTagger

clTag = ClassifierTagger(sents, 'nb')

[Pipeline] .............. (step 1 of 2) Processing vect, total=   2.5s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   1.1s
Tiempo de vectorizacion  + entrenamiento: 3.5602474212646484


In [7]:
# para el eval
files = '3LB-CAST/.*\.tbf\.xml'
corpus_eval = SimpleAncoraCorpusReader('../ancora-3.0.1es', files)
sents_eval = list(corpus_eval.tagged_sents())

In [8]:
clTag.tag_sents(sents_eval)

KeyboardInterrupt: 

In [ ]:
%run ./scripts/train.py -c ../ancora-3.0.1es -m clf -f nb -o nb

In [ ]:
%run ./scripts/eval.py -c ../ancora-3.0.1es -i nb